In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import joblib
from transformers import BertTokenizer, BertForSequenceClassification
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
df=pd.read_excel('C:\\Users\\BIT\\Downloads\\Complain.xlsx')

In [3]:
df.head()

,Domain,Complaint/ Opinion,Complaint Label,Complaint_Cause,Severity level,Sentiment,Emotion
0,Transaction Fail,Can someone tell me if transaction fail betwee...,1,transaction fail between merchant and Bank due...,No explicit reproach,negative,sadness
1,Transaction Fail,@LICIndiaForever @Paytmcare @Paytm As part of ...,1,balance deducted but payment status shows fail,No explicit reproach,negative,sadness
2,Transaction Fail,@ICICIBank_Care This is something very serious...,1,not able to use imobile app,No explicit reproach,negative,anger
3,Transaction Fail,@RBI @ICICIBank_Care @ICICIBank @CRED_support ...,1,payment seems a fraud transaction,Blame,negative,anger
4,Transaction Fail,Fraud transaction through CRED for SBI C-Card ...,1,Fraud transaction through CRED,Accusation,negative,surprise


In [4]:
Lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
    review=re.sub('[^a-zA-Z]',' ',df['Complaint/ Opinion'][i])
    review=review.lower()
    review=review.split()
    review=[Lemmatizer.lemmatize(word , pos='v') for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [5]:
complain_data=pd.DataFrame(corpus)
complain_data.rename(columns={0:"complain"}, inplace=True)

In [6]:
data=pd.concat([complain_data,df['Emotion']],axis=1)

In [7]:
data['Emotion'].unique()

array(['sadness', 'anger', 'surprise', 'fear', 'disgust', 'other',
       'happiness'], dtype=object)

In [8]:
data['Emotion'].replace({'sadness':1, 'anger':2, 'surprise':3, 'fear':4, 'disgust':5, 'other':6,'happiness':7} , inplace=True)

C:\Users\BIT\AppData\Local\Temp\ipykernel_16100\3074045001.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Emotion'].replace({'sadness':1, 'anger':2, 'surprise':3, 'fear':4, 'disgust':5, 'other':6,'happiness':7} , inplace=True)
C:\Users\BIT\AppData\Local\Temp\ipykernel_16100\3074045001.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set

In [12]:
data

,complain,Emotion
0,someone tell transaction fail merchant bank du...,1
1,licindiaforever paytmcare paytm part lic premi...,1
2,icicibank care something serious nd urgent abl...,2
3,rbi icicibank care icicibank cred support theo...,2
4,fraud transaction cred sbi c card payment icic...,3
...,...,...
6275,implication policies reduce emissions save fin...,4
6276,bank put notice regard climate change policies...,7
6277,stock market expect tough year lot opportuniti...,7
6278,goodbye punch bowl go miss federal reserve pre...,5


In [13]:
tfid=TfidfVectorizer(stop_words='english', max_features=2000 , ngram_range=(1,4))
X=data['complain']
Y=data['Emotion']
X_train , X_test , y_train , y_test = train_test_split(X,Y,train_size=0.8 , random_state=42)


In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from scipy.special import softmax

# =======================
# Load dataset
# =======================
df = pd.read_excel("C:\\Users\\BIT\\Downloads\\Complain.xlsx")
df = df[['Complaint/ Opinion', 'Emotion']]

# =======================
# Preprocessing function
# =======================
def preprocess(text):
    new_text = []
    for t in str(text).split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# =======================
# Load model + tokenizer
# =======================
task = "emotion"   # ✅ must be lowercase
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# =======================
# Define labels manually
# =======================
labels = ["anger", "joy", "optimism", "sadness"]

# =======================
# Predict emotions
# =======================
pred_labels = []
pred_scores = []

for text in df["Complaint/ Opinion"]:
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors="pt")
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    label_id = np.argmax(scores)
    pred_labels.append(labels[label_id])
    pred_scores.append(float(scores[label_id]))

# =======================
# Save results
# =======================
df["predicted_emotion"] = pred_labels
df["confidence"] = pred_scores

df.to_csv("complaints_with_predictions.csv", index=False, encoding="utf-8-sig")
print("✅ Predictions saved to complaints_with_predictions.csv")
